In [1]:
from tapi_yandex_metrika import YandexMetrikaLogsapi
from datetime import datetime
import petl as etl

#### Получаем вчерашнюю дату

In [2]:
year = datetime.date(datetime.now()).strftime("%Y")
month = datetime.date(datetime.now()).strftime("%m")
day = str(int(datetime.date(datetime.now()).strftime("%d"))-1)

date = year + '-' + month + '-' + day

#### Указываем Токен и ID счетчика с Я.Метрики

In [3]:
ACCESS_TOKEN = "balblablablablayakrevedko"
COUNTER_ID = "563547"

#### Обращаемся к API, используя токен и ID счетчика

In [4]:
api = YandexMetrikaLogsapi(
    access_token=ACCESS_TOKEN,
    default_url_params={'counterId': COUNTER_ID},
    wait_report=True    
)

#### Задаем параметры для получения двух отчетов: визиты и просмотры страниц

In [5]:
paramsVisit={
    "fields": "ym:s:date,ym:s:clientID,ym:s:lastTrafficSource,ym:s:visitDuration," \
               "ym:s:pageViews,ym:s:isNewUser,ym:s:visitID,ym:s:watchIDs," \
               "ym:s:lastAdvEngine,ym:s:deviceCategory,ym:s:mobilePhone",
    "source": "visits",
    "date1": "2018-01-01",
    "date2": date
}

paramsWatch={
    "fields": "ym:pv:watchID,ym:pv:URL,ym:pv:isPageView,ym:pv:UTMCampaign",
    "source": "hits",
    "date1": "2018-01-01",
    "date2": date
}

#### Оценка возможности получить желаемые отчеты с Я.Метрики

In [6]:
resultV = api.evaluate().get(params=paramsVisit)
resultW = api.evaluate().get(params=paramsWatch)

#### Создание двух отчетов

In [7]:
resultV = api.create().post(params=paramsVisit)
resultW = api.create().post(params=paramsWatch)

#### Присваиваем двум переменным айдишники только что сгенерированных отчетов

In [8]:
requestV_id = resultV().data["log_request"]["request_id"]
requestW_id = resultW().data["log_request"]["request_id"]

#### Скачиваем два сгенерированных ответа по их id 

In [9]:
resultV = api.download(requestId=requestV_id).get()
resultW = api.download(requestId=requestW_id).get()

INFO:root:Включен режим ожидания готовности отчета. Проверка готовности отчета через 20 сек.
INFO:root:Включен режим ожидания готовности отчета. Проверка готовности отчета через 40 сек.
INFO:root:Включен режим ожидания готовности отчета. Проверка готовности отчета через 20 сек.
INFO:root:Включен режим ожидания готовности отчета. Проверка готовности отчета через 40 сек.


#### Кладем данные по визитам и просмотрам в переменные

In [10]:
dataV = resultV().data
dataW = resultW().data

#### Преобразуем данные в списки

In [11]:
dataV = dataV.split('\n')
dataW = dataW.split('\n')

#### Создаем переменные, которые будут использоваться дальше в преобразовании данных

In [12]:
wvTable = [[]]
visitTable = [[]]
watchTable = [[]]
visitArray = []
watchArray = []

#### В переменных *Table для хранения таблиц задаем заголовки таблиц

In [13]:
wvTable[0] = ['VisitID', 'WatchID']

In [14]:
watchTable[0] = [
    'WatchID', #0 <- 0
    'URL', #1 <- 1
    'isPageView', #2 <- 2 
    'UTMCompain' #3 <- 3
]  

In [15]:
visitTable[0] = [
    'Дата', #0 <- 0
    'ClientID', #1 <- 1
    'Источник', #2 <- 2
    'Продолжительность визита', #3 <- 3
    'Количество страниц', #4 <- 4
    'Новый пользователь', #5 <- 5
    'VisitID', #6 <- 6
    'Рекламная система', #7 <- 8
    'Тип устройства', #8 <- 9
    'Тип мобильного устройства', #9 <- 10   
]    

#### Формируем три таблицы с данными

In [16]:
for i in range(1, len(dataV)-1):
    visitArray = dataV[i].split('\t')
    visitTable.append(visitArray[:7] + visitArray[8:])

In [17]:
for i in range(1, len(dataV)-1):
    visitArray = dataV[i].split('\t')
    watchArray = visitArray[7][1:len(visitArray[7])-1].split(',')   
    for watch in watchArray:
        wvTable.append([visitArray[6],watch])  

In [18]:
for i in range(1, len(dataW)-1):
    watchTable.append(dataW[i].split('\t'))

#### Чистим данные и формируем одну общую выходную таблицу

In [19]:
watchTable = etl.leftjoin(watchTable, wvTable, key='WatchID')

In [20]:
watchTable = etl.select(watchTable, lambda rec: rec.VisitID != None)
watchTable = etl.select(watchTable, lambda rec: rec.isPageView != '0')

In [21]:
watchTable = etl.leftjoin(watchTable, visitTable, key='VisitID')

#### Преобразовываем данные в таблице

In [22]:
watchTable = etl.convert(watchTable, 'Тип устройства', 'replace', '1', 'Десктоп')
watchTable = etl.convert(watchTable, 'Тип устройства', 'replace', '2', 'Мобильные телефоны')
watchTable = etl.convert(watchTable, 'Тип устройства', 'replace', '3', 'Планшеты')
watchTable = etl.convert(watchTable, 'Тип устройства', 'replace', '4', 'TV')

In [23]:
watchTable = etl.convert(watchTable, 'Рекламная система', lambda v: v.split('_')[0])

In [24]:
watchTable = etl.convert(watchTable, 'Продолжительность визита', int)
watchTable = etl.convert(watchTable, 'Количество страниц', int)

#### Добавляем дополнительные колонки с данными для нашей конкретной бизнес-задачи

In [25]:
watchTable = etl.addfield(watchTable, 'Длительность просмотра', lambda row: 'меньше 15 сек' 
                          if row['Продолжительность визита'] < 15 
                          else ('от 15 до 30 сек' if row['Продолжительность визита'] >= 15 
                              and row['Продолжительность визита'] < 30 
                                  else ('от 30 до 60 сек' if row['Продолжительность визита'] >= 30 
                                      and row['Продолжительность визита'] < 60 else 'более минуты')))



In [26]:
watchTable = etl.addfield(watchTable, 'Глубина просмотра', lambda row: 'не более 1' 
                          if row['Количество страниц'] <= 1 
                              else ('2-3' if row['Количество страниц'] >= 2 
                                  and row['Количество страниц'] <= 3 
                                      else 'больше 3'))

In [27]:
watchTable = etl.addfield(watchTable, 'Качество трафика', lambda row: 'хорошее' 
                          if (row['Количество страниц'] > 1 
                              and row['Продолжительность визита'] >= 15) 
                                  else ('плохое' if (row['Количество страниц'] <= 1 
                                      and row['Продолжительность визита'] < 15) 
                                          else 'среднее'))

#### Экспортируем в Excel

In [28]:
etl.io.xlsx.toxlsx(watchTable, "oneTable.xlsx", sheet=None, write_header=True)